## Introduction

Throughout this guided project, we'll practice working with different memory constraints.

In this step, let's assume we only have 10 megabytes of available memory.While crunchbase-investments.csv consumes 10.3 megabytes of disk space, we know from earlier missions that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).

In [1]:
import pandas as pd
pd.options.display.max_columns = 99

pd.read_csv('crunchbase-investments.csv', encoding='ISO-8859-1', nrows=7)

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000
5,/company/tackk,Tackk,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-02-01,2012-02,2012-Q1,2012,20000
6,/company/acclaimd,Acclaimd,analytics,USA,OH,Columbus,Columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-06-01,2012-06,2012-Q2,2012,20000


## Missing Columns value counts

Indentify misssing column for each chunk and combine them by columns respectively.

In [2]:
crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
missing_count = []
for chunk in crunch_iter:
    missing_count.append(chunk.isnull().sum())
    
combined_missing = pd.concat(missing_count)
unique_missing = combined_missing.groupby(combined_missing.index).sum()
print(unique_missing.sort_values())

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64


## Memory Footprints

Memory used per each column in MB, and overall memory by default.

In [3]:

crunch_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
memory_footprint = []
for chunk in crunch_iter:
    memory_footprint.append(chunk.memory_usage(deep=True))

memory_combined = pd.concat(memory_footprint)
memory_col = memory_combined.groupby(memory_combined.index).sum()/(1024*1024)
print(memory_col.round(4))
print('\nTotal Memory (MB): ', memory_col.sum().round(4))

Index                     0.0014
company_category_code     3.2626
company_city              3.3435
company_country_code      3.0252
company_name              3.4250
company_permalink         3.8698
company_region            3.2535
company_state_code        2.9622
funded_at                 3.3781
funded_month              3.2268
funded_quarter            3.2268
funded_year               0.4034
funding_round_type        3.2527
investor_category_code    0.5936
investor_city             2.7514
investor_country_code     2.5247
investor_name             3.7343
investor_permalink        4.7498
investor_region           3.2389
investor_state_code       2.3619
raised_amount_usd         0.4034
dtype: float64

Total Memory (MB):  56.989


## Drop Unnecessary Columns

Drop columns representing URL's or containing way too many missing values.

In [13]:

drop_cols = [
'company_permalink',
'company_region',
'investor_permalink',
'investor_region',
'investor_state_code',
'investor_category_code',
'funded_month',
'funded_quarter',
'funded_year']
keep_cols = chunk.columns.drop(drop_cols, errors='ignore')
keep_cols.to_list()


['company_name',
 'company_category_code',
 'company_state_code',
 'company_city',
 'investor_name',
 'investor_country_code',
 'investor_city',
 'funding_round_type',
 'funded_at',
 'raised_amount_usd']

## Inspect string columns candidates for category type

We identify number of unique values by each column. Those with limited unique numbers can be converted to category type to save memory.

In [5]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)
unique_dict = {}
total_row = 0

for chunk in chunk_iter:
    total_row += chunk.shape[0]
    #chunk['company_city'] = chunk['company_city'].str.title()
    chunk_object_cols = chunk.select_dtypes(include=['object']).columns
    for col in chunk_object_cols:
        uniq = chunk[col].value_counts()
        if col in unique_dict:
            unique_dict[col].append(uniq)
        else:
            unique_dict[col] = [uniq]
           
print('column_name' +'\t\t'+ 'unique_values' +'\t\t'+ '% unique')
print('===========================================================')
for col in unique_dict:
    uniq_series = pd.concat(unique_dict[col])
    unique_sum = uniq_series.groupby(uniq_series.index).sum()
    unique_dict[col] = len(unique_sum)
    #if len(unique_sum) < 50: #/42538 < 0.5:
    print((col +'\t'+ str(unique_dict[col])).expandtabs(32) +'\t\t'+ str(round(unique_dict[col]/total_row*100,2)))

column_name		unique_values		% unique
company_name                    11573		21.89
company_category_code           43		0.08
company_country_code            2		0.0
company_state_code              50		0.09
company_city                    1176		2.22
investor_name                   10465		19.79
investor_country_code           72		0.14
investor_city                   990		1.87
funding_round_type              9		0.02
funded_at                       2808		5.31


## Convert Columns to Category, Date an type

- convert columns with limited unique values to category type.
- convert `funded_at` to date type.
- no columns in numeric type. `raised_amount_usd` have 3 missing values hence maintained in `float64` type.
- The column `company_country_code` only have 2 unique values with 1 missing, as mostly represent `USA` we drop this column

In [12]:
keep_cols = keep_cols.drop("company_country_code", errors='ignore')
col_types = {
    "company_category_code": "category", 
    "company_state_code": "category", 
    "company_city": "category",
    "investor_country_code": "category", 
    "investor_state_code": "category",
    "investor_city" : "category",
    "funding_round_type": "category",
}

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols, dtype=col_types, parse_dates=["funded_at"])
overall_memory = 0
for chunk in chunk_iter:
    overall_memory += chunk.memory_usage(deep=True).sum()/(1024**2)

print('Total Columns     : ', chunk.shape[1])  
print('Total Rows        : ', total_row) 
print("Overall Size (MB) : ", overall_memory)

Total Columns     :  10
Total Rows        :  52870
Overall Size (MB) :  9.213824272155762


## Loading chunk into sqlite

Connect to database and create new table `investement`

In [20]:
import sqlite3
conn = sqlite3.connect('crunchbase_investments.db')

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols, dtype=col_types, parse_dates=["funded_at"])
for chunk in chunk_iter:
    chunk.to_sql('investments', conn, if_exists='append', index=False)


In [21]:
pd.read_sql('''SELECT * FROM investments;''', conn)

,company_name,company_category_code,company_state_code,company_city,investor_name,investor_country_code,investor_city,funding_round_type,funded_at,raised_amount_usd
0,AdverCar,advertising,CA,San Francisco,1-800-FLOWERS.COM,USA,New York,series-a,2012-10-30 00:00:00,2000000.0
1,LaunchGram,news,CA,Mountain View,10Xelerator,USA,Columbus,other,2012-01-23 00:00:00,20000.0
2,uTaP,messaging,None,None,10Xelerator,USA,Columbus,other,2012-01-01 00:00:00,20000.0
3,ZoopShop,software,OH,columbus,10Xelerator,USA,Columbus,angel,2012-02-15 00:00:00,20000.0
4,eFuneral,web,OH,Cleveland,10Xelerator,USA,Columbus,other,2011-09-08 00:00:00,20000.0
...,...,...,...,...,...,...,...,...,...,...
52865,Garantia Data,enterprise,CA,Santa Clara,Zohar Gilon,None,None,series-a,2012-08-08 00:00:00,3800000.0
52866,DudaMobile,mobile,CA,Palo Alto,Zohar Gilon,None,None,series-c+,2013-04-08 00:00:00,10300000.0
52867,SiteBrains,software,CA,San Francisco,zohar israel,None,None,angel,2010-08-01 00:00:00,350000.0
52868,Comprehend Systems,enterprise,CA,Palo Alto,Zorba Lieberman,None,None,series-a,2013-07-11 00:00:00,8400000.0


In [24]:
results_df = pd.read_sql('PRAGMA table_info(investments);', conn)
print(results_df)

   cid                   name       type  notnull dflt_value  pk
0    0           company_name       TEXT        0       None   0
1    1  company_category_code       TEXT        0       None   0
2    2     company_state_code       TEXT        0       None   0
3    3           company_city       TEXT        0       None   0
4    4          investor_name       TEXT        0       None   0
5    5  investor_country_code       TEXT        0       None   0
6    6          investor_city       TEXT        0       None   0
7    7     funding_round_type       TEXT        0       None   0
8    8              funded_at  TIMESTAMP        0       None   0
9    9      raised_amount_usd       REAL        0       None   0


## Questions to Answer
1. What proportion of the total amount of funds did the top 10% raise? What about the top 1%?
2. Compare these values to the proportions the bottom 10% and bottom 1% raised.
3. Which category of company attracted the most investments?
4. Which investor contributed the most money (across all startups)?
5. Which investors contributed the most money per startup?
6. Which funding round was the most popular?
7. Which funding round was the least popular?

In [31]:
# 1. What proportion of the total amount of funds did the top 10% raise? What about the top 1%?

pd.read_sql('''
SELECT investor_name,
    SUM(raised_amount_usd) AS total_investment,
    (SUM(raised_amount_usd) / 681732200000) AS '%_of_total'
FROM investments
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
''', conn)

,investor_name,total_investment,%_of_total
0,Kleiner Perkins Caufield & Byers,1.121783e+10,0.016455
1,New Enterprise Associates,9.692542e+09,0.014218
2,Accel Partners,6.472126e+09,0.009494
3,Goldman Sachs,6.375459e+09,0.009352
4,Sequoia Capital,6.039402e+09,0.008859
5,Intel,5.969200e+09,0.008756
6,Google,5.808800e+09,0.008521
7,Time Warner,5.730000e+09,0.008405
8,Comcast,5.669000e+09,0.008316
9,Greylock Partners,4.960983e+09,0.007277


In [32]:
pd.read_sql('''
SELECT investor_name,
    SUM(raised_amount_usd) AS total_investment,
    (SUM(raised_amount_usd) / 681732200000) AS '%_of_total'
FROM investments
GROUP BY 1
ORDER BY 2
LIMIT 10;
''', conn)

,investor_name,total_investment,%_of_total
0,None,None,None
1,1in10 Ventures,None,None
2,212 Capital Partners,None,None
3,2x Consumer Products Growth Partners,None,None
4,313 Ventures,None,None
5,6 angels,None,None
6,7wire Ventures,None,None
7,AA Investments,None,None
8,AEM Capital,None,None
9,AG B,None,None


In [33]:
# 2 Compare values at (1) to the proportions the bottom 10% and bottom 1% raised.

# bottom ten percent of investors
pd.read_sql('''
WITH bottom_ten_investors AS
        (   
        SELECT investor_name,
               SUM(raised_amount_usd) AS total_investment
          FROM investments
         GROUP BY 1
         ORDER BY 2 ASC
         LIMIT 1046
         )

SELECT SUM(tti.total_investment)
  FROM bottom_ten_investors AS tti;
''', conn)

,SUM(tti.total_investment)
0,21409138.0


In [34]:
# bottom one percent of investors
pd.read_sql('''
WITH bottom_one_investors AS
        (   
        SELECT investor_name,
               SUM(raised_amount_usd) AS total_investment
          FROM investments
         GROUP BY 1
         ORDER BY 2 ASC
         LIMIT 105
         )

SELECT SUM(tti.total_investment)
  FROM bottom_one_investors AS tti;
''', conn)

,SUM(tti.total_investment)
0,None


In [36]:
# 3. Which category of company attracted the most investments?
pd.read_sql('''
SELECT company_category_code,
       COUNT(investor_name) AS number_of_investors,
       SUM(raised_amount_usd) AS total_investment
  FROM investments
 GROUP BY 1
 ORDER BY 3 DESC
 LIMIT 1
''', conn)

,company_category_code,number_of_investors,total_investment
0,biotech,4951,1.103964e+11


In [37]:
# 4. Which investor contributed the most money (across all startups)?
pd.read_sql('''
SELECT investor_name,
       SUM(raised_amount_usd) As total_investment
  FROM investments
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 1;
''', conn)

,investor_name,total_investment
0,Kleiner Perkins Caufield & Byers,1.121783e+10


In [39]:
# 5. Which investors contributed the most money per startup?
pd.read_sql('''
SELECT investor_name,
       (raised_amount_usd / company_name) AS money_per_startup
  FROM investments
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 5;
''', conn)

,investor_name,money_per_startup
0,Hillman Ventures,13000000.0
1,Foundry Group,10000000.0
2,O'Reilly AlphaTech Ventures,10000000.0
3,True Ventures,10000000.0
4,KLP Enterprises,5000000.0


In [41]:
# 6. Which funding round was the most popular?
pd.read_sql('''
SELECT funding_round_type,
       COUNT(investor_name)
  FROM investments
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 1;
''', conn)

,funding_round_type,COUNT(investor_name)
0,series-a,13938


In [42]:
# 7. Which funding round was the least popular?
pd.read_sql('''
SELECT funding_round_type,
       COUNT(investor_name)
  FROM investments
 WHERE funding_round_type != 'None'
 GROUP BY 1
 ORDER BY 2 ASC
 LIMIT 1;
''', conn)

,funding_round_type,COUNT(investor_name)
0,crowdfunding,5
